In [2]:
# 모듈 import
import time
import sys 
import os 
import re

import pandas as pd
import numpy as np

from selenium import webdriver 
# import chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains


import urllib.request as req
from bs4 import BeautifulSoup


import warnings
warnings.filterwarnings('ignore')

# 경로 지정
- 시작시 변경 필요!!!!

In [3]:
driver_path = "C:/chromedriver.exe"
csv_path = "C:/users/jinyo/"

os.chdir(csv_path)

# 검색 지역 설정

## 지역명 DB 불러오기

In [4]:
loc_df = pd.read_csv('korea_location.csv')
loc_df[:10]

,location_no,location,place1_no,place1,place2_no,place2,place
0,11,서울특별시,11010,종로구,11010720,청운효자동,종로구 청운효자동
1,11,서울특별시,11010,종로구,11010530,사직동,종로구 사직동
2,11,서울특별시,11010,종로구,11010540,삼청동,종로구 삼청동
3,11,서울특별시,11010,종로구,11010550,부암동,종로구 부암동
4,11,서울특별시,11010,종로구,11010560,평창동,종로구 평창동
5,11,서울특별시,11010,종로구,11010570,무악동,종로구 무악동
6,11,서울특별시,11010,종로구,11010580,교남동,종로구 교남동
7,11,서울특별시,11010,종로구,11010600,가회동,종로구 가회동
8,11,서울특별시,11010,종로구,11010610,종로1.2.3.4가동,종로구 종로1.2.3.4가동
9,11,서울특별시,11010,종로구,11010630,종로5.6가동,종로구 종로5.6가동


## 지역 내 카페목록 가져오기

In [5]:
# 리스트변수명 설정
driver = webdriver.Chrome(driver_path)
place_list=[]
title_list = []
f_data_list = []

# 수집
for p in range(10):   # ========================================================================================== 테스트용
#for p in range(len(loc_df)): 
    # 검색어 입력    
    query = '{0} 카페'.format(loc_df['place'][p]) #------------지역[칼럼명]으로 데이터마다 수정
    driver.get(f"https://map.naver.com/v5/search/{query}?c=14203933.7141038,4562681.4505997,10,0,0,0,dh")
    time.sleep(2)
    # 검색결과 iframe 접근
    driver.switch_to.frame("searchIframe")
    
    # 처음 돌릴 땐 자료 60개 수집
    # 1페이지로 돌아간 뒤 한번 더 돌리면 300개 정도 추출됨 (검색 결과 많은 경우)

    try: 
        for i in range(1,7): 
            driver.find_element_by_link_text(str(i)).click()
            time.sleep(2)
            
            try: 
                for j in range(3,70,3):
                    element1 = driver.find_elements_by_css_selector('.OXiLu')[j]
                    ActionChains(driver).move_to_element(element1).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
                    element2 = driver.find_elements_by_css_selector('._3Apve')[j]
                    ActionChains(driver).move_to_element(element2).key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
            except:
                pass
            
                
            # 카페명
            if len(driver.find_elements_by_css_selector('.OXiLu')) == 0:
                title_raw = driver.find_elements_by_css_selector('._3Apve')
            else:
                title_raw = driver.find_elements_by_css_selector('.OXiLu')
            for title in title_raw:
                title = title.text
                place_list.append(loc_df['place'][p]) #------------지역[칼럼명]으로 데이터마다 수정
                title_list.append(title)
            
            
#             print(i,'p',end='\t') #몇번째 페이지 크롤링 중인지 확인
    except:
        pass
    print(query,"/ 누적 카페 수 :", len(title_list) )

1,2,3,4,5,6,종로구 청운효자동 카페 / 누적 카페 수 : 60
1,2,3,4,5,6,종로구 사직동 카페 / 누적 카페 수 : 120
1,2,3,4,5,6,종로구 삼청동 카페 / 누적 카페 수 : 180
1,2,3,4,5,6,종로구 부암동 카페 / 누적 카페 수 : 240
1,2,3,4,5,종로구 평창동 카페 / 누적 카페 수 : 280
1,2,3,4,5,6,종로구 무악동 카페 / 누적 카페 수 : 340
1,2,3,4,5,6,종로구 교남동 카페 / 누적 카페 수 : 400
1,2,3,4,5,6,종로구 가회동 카페 / 누적 카페 수 : 460
1,2,3,4,5,6,종로구 종로1.2.3.4가동 카페 / 누적 카페 수 : 520
1,2,3,4,5,6,종로구 종로5.6가동 카페 / 누적 카페 수 : 580


In [6]:
## 데이터 프레임 만들기
list_df = pd.DataFrame({'place':place_list, 'title':title_list})
list_df

,place,title
0,종로구 청운효자동,더숲 초소책방
1,종로구 청운효자동,에디션덴마크쇼룸
2,종로구 청운효자동,아키비스트
3,종로구 청운효자동,서촌스코프
4,종로구 청운효자동,효자베이커리
...,...,...
575,종로구 종로5.6가동,미러볼커피스탠드
576,종로구 종로5.6가동,러셔스 스쿱스
577,종로구 종로5.6가동,을지다락
578,종로구 종로5.6가동,레레플레이


In [7]:
## csv 저장 
list_df.to_csv(csv_path+"cafe_list.csv", encoding='cp949') 

# 저장 후 카페 리스트에 접근

In [8]:
## 카페 리스트 load
cafe_df = pd.read_csv(csv_path+"cafe_list.csv", encoding='cp949')
cafe_df

,Unnamed: 0,place,title
0,0,종로구 청운효자동,더숲 초소책방
1,1,종로구 청운효자동,에디션덴마크쇼룸
2,2,종로구 청운효자동,아키비스트
3,3,종로구 청운효자동,서촌스코프
4,4,종로구 청운효자동,효자베이커리
...,...,...,...
575,575,종로구 종로5.6가동,미러볼커피스탠드
576,576,종로구 종로5.6가동,러셔스 스쿱스
577,577,종로구 종로5.6가동,을지다락
578,578,종로구 종로5.6가동,레레플레이


In [9]:
# url 담을 컬럼 만들어놓기
cafe_df['search'] = cafe_df['place']+' '+cafe_df['title']
cafe_df['url'] = '' 
cafe_df.drop('Unnamed: 0', axis = 1, inplace = True)
cafe_df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)
cafe_df

,place,title,search,url
0,종로구 청운효자동,더숲 초소책방,종로구 청운효자동 더숲 초소책방,
1,종로구 청운효자동,에디션덴마크쇼룸,종로구 청운효자동 에디션덴마크쇼룸,
2,종로구 청운효자동,아키비스트,종로구 청운효자동 아키비스트,
3,종로구 청운효자동,서촌스코프,종로구 청운효자동 서촌스코프,
4,종로구 청운효자동,효자베이커리,종로구 청운효자동 효자베이커리,
...,...,...,...,...
575,종로구 종로5.6가동,미러볼커피스탠드,종로구 종로5.6가동 미러볼커피스탠드,
576,종로구 종로5.6가동,러셔스 스쿱스,종로구 종로5.6가동 러셔스 스쿱스,
577,종로구 종로5.6가동,을지다락,종로구 종로5.6가동 을지다락,
578,종로구 종로5.6가동,레레플레이,종로구 종로5.6가동 레레플레이,


# url 찾기 for문

In [97]:
#### 여러개가 나오면, url을 가져올 수 없음

In [10]:
for i, keyword in enumerate(cafe_df['search'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {cafe_df.shape[0]} 행", keyword)
    try:
        naver_map_search_url = f'https://map.naver.com/v5/search/+{keyword}/place' # 검색 url 만들기
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기 
        time.sleep(4)  # 중요함, 중간에 500에러 발생 방지 위함 
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지 
        res_code = re.findall(r"place/(\d+)", cu)
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#'
        print(final_url)
        cafe_df['url'][i]=final_url
    except IndexError: 
        cafe_df['url'][i]= '' 
        print('none') 
print("-"*30, '종료', '-'*30)

이번에 찾을 키워드 : 0 / 579 행 종로구 청운효자동 더숲 초소책방
https://pcmap.place.naver.com/restaurant/1602048207/review/visitor#
이번에 찾을 키워드 : 1 / 579 행 종로구 청운효자동 에디션덴마크쇼룸
https://pcmap.place.naver.com/restaurant/1787227960/review/visitor#
이번에 찾을 키워드 : 2 / 579 행 종로구 청운효자동 아키비스트
https://pcmap.place.naver.com/restaurant/1783153281/review/visitor#
이번에 찾을 키워드 : 3 / 579 행 종로구 청운효자동 서촌스코프
https://pcmap.place.naver.com/restaurant/462568628/review/visitor#
이번에 찾을 키워드 : 4 / 579 행 종로구 청운효자동 효자베이커리
https://pcmap.place.naver.com/restaurant/17898357/review/visitor#
이번에 찾을 키워드 : 5 / 579 행 종로구 청운효자동 통인스윗
none
이번에 찾을 키워드 : 6 / 579 행 종로구 청운효자동 베어카페
none
이번에 찾을 키워드 : 7 / 579 행 종로구 청운효자동 카페이마
https://pcmap.place.naver.com/restaurant/11678597/review/visitor#
이번에 찾을 키워드 : 8 / 579 행 종로구 청운효자동 인왕산 대충유원지
https://pcmap.place.naver.com/restaurant/1077382386/review/visitor#
이번에 찾을 키워드 : 9 / 579 행 종로구 청운효자동 스쿠퍼젤라또
https://pcmap.place.naver.com/restaurant/1553059011/review/visitor#
이번에 찾을 키워드 : 10 / 579 행 종로구 청운효자동 할리스 세종로점
none
이번에 찾

https://pcmap.place.naver.com/restaurant/31481759/review/visitor#
이번에 찾을 키워드 : 89 / 579 행 종로구 사직동 커피빈 경희궁의아침점
none
이번에 찾을 키워드 : 90 / 579 행 종로구 사직동 카페자스 광화문점
none
이번에 찾을 키워드 : 91 / 579 행 종로구 사직동 Simple 광화문점
https://pcmap.place.naver.com/restaurant/495850859/review/visitor#
이번에 찾을 키워드 : 92 / 579 행 종로구 사직동 카페큐브
https://pcmap.place.naver.com/restaurant/38695592/review/visitor#
이번에 찾을 키워드 : 93 / 579 행 종로구 사직동 와프
https://pcmap.place.naver.com/restaurant/1754811858/review/visitor#
이번에 찾을 키워드 : 94 / 579 행 종로구 사직동 히든커피
https://pcmap.place.naver.com/restaurant/1772406863/review/visitor#
이번에 찾을 키워드 : 95 / 579 행 종로구 사직동 커피에반하다 종로도렴빌딩점 스마트카페
https://pcmap.place.naver.com/restaurant/1405650173/review/visitor#
이번에 찾을 키워드 : 96 / 579 행 종로구 사직동 슬로우레시피
https://pcmap.place.naver.com/restaurant/1410501334/review/visitor#
이번에 찾을 키워드 : 97 / 579 행 종로구 사직동 카페광화문
none
이번에 찾을 키워드 : 98 / 579 행 종로구 사직동 오늘도다온치즈
https://pcmap.place.naver.com/restaurant/1184967116/review/visitor#
이번에 찾을 키워드 : 99 / 579 행 종로구 사직동 사직로 커

https://pcmap.place.naver.com/restaurant/13151950/review/visitor#
이번에 찾을 키워드 : 178 / 579 행 종로구 삼청동 앞으로의빵집
https://pcmap.place.naver.com/restaurant/1525340544/review/visitor#
이번에 찾을 키워드 : 179 / 579 행 종로구 부암동 부빙
none
이번에 찾을 키워드 : 180 / 579 행 종로구 부암동 묘한빵집
https://pcmap.place.naver.com/restaurant/1615875960/review/visitor#
이번에 찾을 키워드 : 181 / 579 행 종로구 부암동 산모퉁이
https://pcmap.place.naver.com/restaurant/12781756/review/visitor#
이번에 찾을 키워드 : 182 / 579 행 종로구 부암동 부암동스코프
https://pcmap.place.naver.com/restaurant/37844910/review/visitor#
이번에 찾을 키워드 : 183 / 579 행 종로구 부암동 부암동 카페 스탐티쉬
https://pcmap.place.naver.com/restaurant/19882640/review/visitor#
이번에 찾을 키워드 : 184 / 579 행 종로구 부암동 클럽에스프레소
https://pcmap.place.naver.com/restaurant/11694272/review/visitor#
이번에 찾을 키워드 : 185 / 579 행 종로구 부암동 부암동 몽유도원 도이창
https://pcmap.place.naver.com/restaurant/1107873485/review/visitor#
이번에 찾을 키워드 : 186 / 579 행 종로구 부암동 몽핀
https://pcmap.place.naver.com/restaurant/1358846917/review/visitor#
이번에 찾을 키워드 : 187 / 579 행 종로구 부암동 

none
이번에 찾을 키워드 : 262 / 579 행 종로구 평창동 파리바게뜨 성대명륜점
none
이번에 찾을 키워드 : 263 / 579 행 종로구 평창동 헤이마
https://pcmap.place.naver.com/restaurant/37902818/review/visitor#
이번에 찾을 키워드 : 264 / 579 행 종로구 평창동 갤러리 커피하우스
https://pcmap.place.naver.com/restaurant/49949556/review/visitor#
이번에 찾을 키워드 : 265 / 579 행 종로구 평창동 카페350
none
이번에 찾을 키워드 : 266 / 579 행 종로구 평창동 밀롱가
https://pcmap.place.naver.com/restaurant/37879725/review/visitor#
이번에 찾을 키워드 : 267 / 579 행 종로구 평창동 업스테어카페
https://pcmap.place.naver.com/restaurant/610071494/review/visitor#
이번에 찾을 키워드 : 268 / 579 행 종로구 평창동 꾸띠
https://pcmap.place.naver.com/restaurant/1642426468/review/visitor#
이번에 찾을 키워드 : 269 / 579 행 종로구 평창동 안다미로상명대자하관점
https://pcmap.place.naver.com/restaurant/36741561/review/visitor#
이번에 찾을 키워드 : 270 / 579 행 종로구 평창동 마티네
https://pcmap.place.naver.com/restaurant/89041677/review/visitor#
이번에 찾을 키워드 : 271 / 579 행 종로구 평창동 레브
https://pcmap.place.naver.com/restaurant/1478522268/review/visitor#
이번에 찾을 키워드 : 272 / 579 행 종로구 평창동 카페SMC 상명대점
https://pcmap

https://pcmap.place.naver.com/restaurant/37996721/review/visitor#
이번에 찾을 키워드 : 353 / 579 행 종로구 교남동 모브
https://pcmap.place.naver.com/restaurant/1187495648/review/visitor#
이번에 찾을 키워드 : 354 / 579 행 종로구 교남동 통의동단팥
https://pcmap.place.naver.com/restaurant/35272163/review/visitor#
이번에 찾을 키워드 : 355 / 579 행 종로구 교남동 라그린
https://pcmap.place.naver.com/restaurant/1310154588/review/visitor#
이번에 찾을 키워드 : 356 / 579 행 종로구 교남동 세이지핀치
https://pcmap.place.naver.com/restaurant/1276434255/review/visitor#
이번에 찾을 키워드 : 357 / 579 행 종로구 교남동 베이커스퍼센트
https://pcmap.place.naver.com/restaurant/1242831532/review/visitor#
이번에 찾을 키워드 : 358 / 579 행 종로구 교남동 궤도
none
이번에 찾을 키워드 : 359 / 579 행 종로구 교남동 커피원
https://pcmap.place.naver.com/restaurant/267047603/review/visitor#
이번에 찾을 키워드 : 360 / 579 행 종로구 교남동 onground
https://pcmap.place.naver.com/restaurant/32520926/review/visitor#
이번에 찾을 키워드 : 361 / 579 행 종로구 교남동 마샹스
https://pcmap.place.naver.com/restaurant/749606138/review/visitor#
이번에 찾을 키워드 : 362 / 579 행 종로구 교남동 커피한잔
none
이번에 

https://pcmap.place.naver.com/restaurant/1120415606/review/visitor#
이번에 찾을 키워드 : 448 / 579 행 종로구 가회동 츄로101 삼청점
https://pcmap.place.naver.com/restaurant/37491896/review/visitor#
이번에 찾을 키워드 : 449 / 579 행 종로구 가회동 스프링포니
https://pcmap.place.naver.com/restaurant/1498933893/review/visitor#
이번에 찾을 키워드 : 450 / 579 행 종로구 가회동 브리오슈도레 그랑서울점
https://pcmap.place.naver.com/restaurant/37867443/review/visitor#
이번에 찾을 키워드 : 451 / 579 행 종로구 가회동 우즈베이커리 익선점
https://pcmap.place.naver.com/restaurant/1365860703/review/visitor#
이번에 찾을 키워드 : 452 / 579 행 종로구 가회동 카페종로시장
https://pcmap.place.naver.com/restaurant/1063972078/review/visitor#
이번에 찾을 키워드 : 453 / 579 행 종로구 가회동 카페기억
https://pcmap.place.naver.com/restaurant/1037859057/review/visitor#
이번에 찾을 키워드 : 454 / 579 행 종로구 가회동 담쟁이집
https://pcmap.place.naver.com/restaurant/1008202610/review/visitor#
이번에 찾을 키워드 : 455 / 579 행 종로구 가회동 아티제 종로그랑서울2점
none
이번에 찾을 키워드 : 456 / 579 행 종로구 가회동 CFlamingo
https://pcmap.place.naver.com/restaurant/35457966/review/visitor#
이번에 찾을 키워드 :

none
이번에 찾을 키워드 : 537 / 579 행 종로구 종로5.6가동 카페 클로이
https://pcmap.place.naver.com/restaurant/1808908687/review/visitor#
이번에 찾을 키워드 : 538 / 579 행 종로구 종로5.6가동 리카페
none
이번에 찾을 키워드 : 539 / 579 행 종로구 종로5.6가동 숏컷 익스프레스
https://pcmap.place.naver.com/restaurant/1628354351/review/visitor#
이번에 찾을 키워드 : 540 / 579 행 종로구 종로5.6가동 동대문커피팩토리
https://pcmap.place.naver.com/restaurant/1228586797/review/visitor#
이번에 찾을 키워드 : 541 / 579 행 종로구 종로5.6가동 커피휴
https://pcmap.place.naver.com/restaurant/1910436464/review/visitor#
이번에 찾을 키워드 : 542 / 579 행 종로구 종로5.6가동 가발카페 동대문점
https://pcmap.place.naver.com/restaurant/1260802004/review/visitor#
이번에 찾을 키워드 : 543 / 579 행 종로구 종로5.6가동 사단법인 길동무
https://pcmap.place.naver.com/restaurant/1622998695/review/visitor#
이번에 찾을 키워드 : 544 / 579 행 종로구 종로5.6가동 커피&디저트카페
none
이번에 찾을 키워드 : 545 / 579 행 종로구 종로5.6가동 종로다방
none
이번에 찾을 키워드 : 546 / 579 행 종로구 종로5.6가동 코르코르디움
https://pcmap.place.naver.com/restaurant/1908696393/review/visitor#
이번에 찾을 키워드 : 547 / 579 행 종로구 종로5.6가동 델리샵
none
이번에 찾을 키워드 : 54

In [11]:
cafe_df.to_csv('cafe_link.csv', encoding = 'utf-8-sig')

In [1]:
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#
#=================================================================================================================#

# 리뷰 크롤링

In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time, re, os
from bs4 import BeautifulSoup
from tqdm import tqdm 

In [10]:
driver_path = "C:/chromedriver.exe"
csv_path = "C:/users/jinyo/"

os.chdir(csv_path)

In [6]:
df = pd.read_csv('cafe_link.csv') 
# 빈칸 제외
df['url'].replace('', np.nan, inplace=True)
df = df.dropna()
df.drop('Unnamed: 0', axis = 1, inplace = True)
print(len(df))
df = df.reset_index()
df

421


,index,place,title,search,url
0,0,종로구 청운효자동,더숲 초소책방,종로구 청운효자동 더숲 초소책방,https://pcmap.place.naver.com/restaurant/16020...
1,1,종로구 청운효자동,에디션덴마크쇼룸,종로구 청운효자동 에디션덴마크쇼룸,https://pcmap.place.naver.com/restaurant/17872...
2,2,종로구 청운효자동,아키비스트,종로구 청운효자동 아키비스트,https://pcmap.place.naver.com/restaurant/17831...
3,3,종로구 청운효자동,서촌스코프,종로구 청운효자동 서촌스코프,https://pcmap.place.naver.com/restaurant/46256...
4,4,종로구 청운효자동,효자베이커리,종로구 청운효자동 효자베이커리,https://pcmap.place.naver.com/restaurant/17898...
...,...,...,...,...,...
416,572,종로구 종로5.6가동,카페 빵쌤,종로구 종로5.6가동 카페 빵쌤,https://pcmap.place.naver.com/restaurant/20300...
417,573,종로구 종로5.6가동,이공오도씨,종로구 종로5.6가동 이공오도씨,https://pcmap.place.naver.com/restaurant/17716...
418,574,종로구 종로5.6가동,미러볼커피스탠드,종로구 종로5.6가동 미러볼커피스탠드,https://pcmap.place.naver.com/restaurant/16031...
419,576,종로구 종로5.6가동,을지다락,종로구 종로5.6가동 을지다락,https://pcmap.place.naver.com/restaurant/17331...


## 데이터프레임 만든 후 저장

In [12]:
# 웹드라이버 접속 
driver = webdriver.Chrome(driver_path)

C:\Users\jinyo\AppData\Local\Temp\ipykernel_20768\4137774923.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [13]:
# 크롤링
cafe_li = []
review_li = []
tag_li = []

print("총 카페수", len(df['title']))
print("-"*10,"Tag 크롤링 시작", "-"*10)

for j in range(2):     # ========================================================================================== 테스트용
# for j in range(len(df)):
    driver.get(df['url'][j]) 
    for i in range(10):
        # 버튼 누르기
        while True:
            try:
                more_tags = driver.find_element_by_css_selector('a.utrsf._30ord')
                more_tags.send_keys(Keys.ENTER)
            except:
                break
        reviews = driver.find_elements_by_css_selector('li:nth-child({0}) > div.faZHB > a > span.WoYOw'.format(i))
        review_tags = driver.find_elements_by_css_selector('li:nth-child({0}) > div._1aSvN > span.utrsf'.format(i))
        for review in reviews:
            cafe_li.append(df['title'][j])
            review_li.append(review.text)
            tags = []
            for tag in review_tags:
                tags.append(tag.text)
            tag_li.append('&'.join(tags))  
    print(j+1,"/",len(df['title']),"\t","********** [{}] 리뷰 tag added **********".format(df['title'][j]))
print("-"*10,"Tag 크롤링 종료", "-"*10)

총 카페수 421
---------- Tag 크롤링 시작 ----------


C:\Users\jinyo\AppData\Local\Temp\ipykernel_20768\589893243.py:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  more_tags = driver.find_element_by_css_selector('a.utrsf._30ord')
C:\Users\jinyo\AppData\Local\Temp\ipykernel_20768\589893243.py:20: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  reviews = driver.find_elements_by_css_selector('li:nth-child({0}) > div.faZHB > a > span.WoYOw'.format(i))
C:\Users\jinyo\AppData\Local\Temp\ipykernel_20768\589893243.py:21: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  review_tags = driver.find_elements_by_css_selector('li:nth-child({0}) > div._1aSvN > span.utrsf'.format(i))


1 / 421 	 ********** [더숲 초소책방] 리뷰 tag added **********
2 / 421 	 ********** [에디션덴마크쇼룸] 리뷰 tag added **********
---------- Tag 크롤링 종료 ----------


In [15]:
review_tag_df = pd.DataFrame((zip(cafe_li, review_li, tag_li)), columns = ['cafe', 'review', 'tag'])
review_tag_df

,cafe,review,tag
0,더숲 초소책방,사장님이 정말 친절하세요 ㅠ 감동 수준;; 바람이 심하게 날리는 편입니다 파라솔이 ...,친절해요&사진이 잘 나와요&집중하기 좋아요&뷰가 좋아요&디저트가 맛있어요
1,더숲 초소책방,예전 초소를 멋지게 바꾸어 카페를 만든 곳이예요~워낙 유명한 곳이라 평일에도 차들과...,커피가 맛있어요&디저트가 맛있어요&뷰가 좋아요&대화하기 좋아요&친절해요
2,더숲 초소책방,방송을 타서 그런지 정말 사람이 많았아요 왠만하면 오후 늦게 가시길 추천. 빵 맛집...,디저트가 맛있어요&뷰가 좋아요
3,더숲 초소책방,뷰가 좋아 조용히 오전중으로 책 읽고 오려다가 사람이 너무 많아.. 구석진 바위에 ...,커피가 맛있어요&디저트가 맛있어요&뷰가 좋아요&읽을만한 책이 많아요
4,더숲 초소책방,집 근처라서 자주 가는 곳 이긴한데~갈때마다 이쁘네요~주말엔 사람도 많고 북적이긴한...,사진이 잘 나와요&뷰가 좋아요&대화하기 좋아요&좌석이 편해요
5,더숲 초소책방,뷰맛집 ㅎㅎ 방송 타고 사람이 더 많아져서 아쉽지만.. 날씨 좋을 시기에 와보고 싶...,커피가 맛있어요&인테리어가 멋져요&뷰가 좋아요&사진이 잘 나와요
6,더숲 초소책방,인왕산 숲길이나 자락길 산책하면서 꼭 가보세요. 서울뷰 이렇게 예쁘게 잘 보이는 곳...,커피가 맛있어요&디저트가 맛있어요&인테리어가 멋져요&사진이 잘 나와요&뷰가 좋아요
7,더숲 초소책방,주차도와주시는 분이 이중주차시 도와주시네요. 전 도착해서 막 나가는 차가 있어 운좋...,좌석이 편해요&친절해요&디저트가 맛있어요&음료가 맛있어요&뷰가 좋아요
8,더숲 초소책방,평일 아침-낮에 가면 사람이 적어서 뷰 즐기기 좋아요. 빵 가격대가 높은 편이지만 ...,커피가 맛있어요&디저트가 맛있어요&사진이 잘 나와요&뷰가 좋아요&읽을만한 책이 많아요
9,에디션덴마크쇼룸,북유럽에서 유명한 커피컬렉티브 커피를 맛볼수 있다는 기대감에 설레이며 방문해서 일단...,커피가 맛있어요&디저트가 맛있어요&친절해요


In [16]:
review_tag_df.to_csv(csv_path+"review_tag_df.csv",
#            sep='\t', 
           encoding='utf-8-sig')